<table style="width: 100%; border-style: none">
 <tbody>
  <tr style="border-style: none">
   <td style="border-style: none; width: 1%; text-align: left; font-size: 16px">Institute for Biomedical Imaging<br />Hamburg University of Technology</td>
   <td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
   <td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Dr. Martin Möddel<br />Mirco Grosser<br />Marija Boberg</td>
  </tr>
 </tbody>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computer Graphics</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Exercise 3 - Getting Started with OpenGL</h1>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">summer term 2023</h3>
<hr>

## 0. Get started OpenGL
For this exercise you need to install the julia packages `ModernGL` and `GLFW` via the package manager with `] add ModernGL GLFW`. For GLFW you need to install the following packages first:
* Linux: sudo apt install cmake libxrandr-dev libxinerama-dev libxcursor-dev
* Windows/MacOS: install cmake from https://cmake.org/

Your grafic card drivers need to support the minimum OpenGL v.3.2 so you need to check for that.
* Linux: glxinfo | grep "version"
* Windows: https://support.pix4d.com/hc/en-us/articles/203876689-How-to-verify-that-the-Driver-of-the-Graphics-Card-supports-OpenGL-3-2

Most code snippets and examples are taken from existing tutorials and can be found here.
* https://open.gl/
* https://learnopengl.com/
* http://www.opengl-tutorial.org/beginners-tutorials/

Documentations:
* GLFW http://www.glfw.org/documentation.html
* OpenGL https://www.khronos.org/registry/OpenGL-Refpages/gl4/


## 1. Setting up a window and event-loop using GLFW

Before working with OpenGL, we need to care of the creation of a window and OpenGL context (basically a state maschine holding all data related to rendering).
The creation of a window and OpenGL context is platform dependent and not part of the OpenGL specification. Therefore, we use abstraction libraries (GLFW in our case) to handle window creation, OpenGL context creation and input handling. This lets us focus on the OpenGL application itself.

The following code imports the library, creates a window and opens an event loop which does nothing but swap the back- and front buffer periodically, without drawing anything. Furthermore, we add a call-back function, such that the window closes when the `esc` button is pressed. 

Since the setup will be similar in the following examples, we wrap the window generation in a separate function.

In [11]:
import GLFW  # we focus exclusively on the GLFW approach
using ModernGL

# create a window and a corresponding OpenGL context
function initWindow(;windowSize::NTuple{2,Int64}=(800,600), title::String="window 1", callbackFunc=key_callback)

    # The GLFW package calls GLFW.Init() automatically when loaded

    # Specify minimum versions
    GLFW.WindowHint(GLFW.CONTEXT_VERSION_MAJOR, 3)  # minimum OpenGL v. 3
    GLFW.WindowHint(GLFW.CONTEXT_VERSION_MINOR, 2)  # minimum OpenGL v. 3.2
    GLFW.WindowHint(GLFW.OPENGL_PROFILE, GLFW.OPENGL_CORE_PROFILE)
    GLFW.WindowHint(GLFW.OPENGL_FORWARD_COMPAT, GL_TRUE)

    # Create the window
    GLFW.WindowHint(GLFW.RESIZABLE, GL_FALSE)
    window = GLFW.CreateWindow(windowSize[1], windowSize[2], title)  # windowed
    GLFW.MakeContextCurrent(window)

    # This is a touch added from
    #   http://www.opengl-tutorial.org/beginners-tutorials/tutorial-1-opening-a-window/:
    # Retain keypress events until the next call to GLFW.GetKey, even if
    # the key has been released in the meantime
    GLFW.SetInputMode(window, GLFW.STICKY_KEYS, GL_TRUE)

    GLFW.SetKeyCallback(window,callbackFunc)

    return window
end

function key_callback(window::GLFW.Window, key::GLFW.Key, scancode::Int32, action::GLFW.Action, mods::Int32)
    # close window when the esc-key is pressed
    if (key == GLFW.KEY_ESCAPE && action == GLFW.PRESS)
        GLFW.SetWindowShouldClose(window, true)
    end
end

window = initWindow(title="1. Window", callbackFunc=key_callback)

# Draw nothing, while waiting for a close event
while !GLFW.WindowShouldClose(window)
    
    # Render here
    glClear(GL_COLOR_BUFFER_BIT)
    
    # Swap front and back buffers
    GLFW.SwapBuffers(window)
    
    # Poll for and process events
    GLFW.PollEvents()    
end
GLFW.DestroyWindow(window)  # needed if you're running this from the REPL
# GLFW.Terminate() is called automatically upon exit

Dict{GLFW.Window, Ref{Vector{Union{Nothing, Function}}}}()

## 2. Our First Triangle

Let us draw a simple triangle. This requires us to do the following things:
* Implement a basic vertex shader and a fragment shader
* Compile the shader programs
* Store the vertices of the triangle in a Vertex Buffer Object (VBO)
* Generate a Vertex Array Object (VAO) and bind the VBO to it
* Modify the event-loop such that the object is drawn in each iteration

Let us start with the setting up a window and OpenGL context just as we did before

In [12]:
window = initWindow(title="Triangle", callbackFunc=key_callback)

GLFW.Window(Ptr{Nothing} @0x00000000052aa290)

The vertex shader and fragment shader are written in the OpenGL Shader Language (GLSL). 
To store the sorce code, we use triple quoted string literals, as in `vertex_source = """..."""`.
More informations on the types supported by GLSL can be found here: http://learnwebgl.brown37.net/12_shader_language/glsl_data_types.html.

Here, we are going to implement the simplest shaders possible.
* The input to the vertex shader is the 2d-coordinate of each vertex. The vertex shader then sets these as the x- and y-coordinates of the 4-Vector `gl_Position`.
* The output of the fragment shader determines the output color for each fragment. In this case we simply return the same color for all fragments.

In [13]:
### implement vertex and fragment shader in GLSL (OpenGL Shading Language) ###

# The vertex shader
vertex_source = """
#version 150
in vec2 position;
void main()
{
    gl_Position = vec4(position, 0.0, 1.0);
}
"""

# The fragment shader #(1.0, 0.0, 0.0, 1.0)
fragment_source = """
# version 150
out vec4 outColor;
void main()
{
    outColor = vec4(0.176, 0.776, 0.839, 1.0);
}
"""

"# version 150\nout vec4 outColor;\nvoid main()\n{\n    outColor = vec4(0.176, 0.776, 0.839, 1.0);\n}\n"

Next, let us compile the shaders and form the shader program. Again, we wrap this into a separate function, because this code snippet occurs in all of the following examples.

In [14]:
function compileShaderProgram(vertex_source::String, fragment_source::String)
   # Compile the vertex shader
    vertex_shader = glCreateShader(GL_VERTEX_SHADER)
    vertex_shader_Ptr = Ptr{UInt8}[pointer(vertex_source)]
    lenVert = Ref{GLint}(length(vertex_source))
    glShaderSource(vertex_shader,1, vertex_shader_Ptr, lenVert)
    glCompileShader(vertex_shader)

    # Check for compilation errors
    status = Ref(GLint(0))
    glGetShaderiv(vertex_shader, GL_COMPILE_STATUS, status)
    if status[] != GL_TRUE
        buffer = Array(UInt8, 512)
        glGetShaderInfoLog(vertex_shader, 512, C_NULL, buffer)
        error(unsafe_string(buffer))
    end

    # Compile the fragment shader
    fragment_shader = glCreateShader(GL_FRAGMENT_SHADER)
    fragment_shader_Ptr = Ptr{UInt8}[pointer(fragment_source)]
    lenFrag = Ref{GLint}(length(fragment_source))
    glShaderSource(fragment_shader, 1, fragment_shader_Ptr, lenFrag)
    glCompileShader(fragment_shader)

    #Check for compilation errors
    statusFrag=Ref(GLint(0))
    glGetShaderiv(fragment_shader, GL_COMPILE_STATUS, statusFrag)
    if statusFrag[] != GL_TRUE
        buffer = Array(UInt8,512)
        glGetShaderInfoLog(fragment_shader, 512, C_NULL, buffer)
        error(unsafe_string(buffer))
    end

    shader_program = glCreateProgram();
    glAttachShader(shader_program, vertex_shader);
    glAttachShader(shader_program, fragment_shader);

    glLinkProgram(shader_program)
    glUseProgram(shader_program)

    return shader_program
end

shader_program = compileShaderProgram(vertex_source, fragment_source)

0x00000003

Let us define a triangle with vertices (0,0.5), (0.5,-0.5) and (-0.5,-0.5). In order to store them in a VBO, we store the vertex coordinates in a `Vector{Float32}`.
The following lines of code generate a VBO and assign the vertex data to it.

Next we need to generate the VAO and make it current.
The VAO is used to link the vertex data to the attributes in the shader programs. This can be done in different places. However, the VAO needs be made current before calling `glVertexAttribPointer` and before creating any Element Buffer Objects.

In [15]:
vertices = [0.0f0,0.5f0, 0.5f0,-0.5f0, -0.5f0,-0.5f0] # note Float32

# Vertex Buffer Object (VBO)
vbo = Ref(GLuint(0))   # initial value is irrelevant, just allocate space
glGenBuffers(1, vbo)   # generate one Buffer object name and store it in vbo
glBindBuffer(GL_ARRAY_BUFFER, vbo[])   # bind the buffer to target GL_ARRAY_BUFFER (i.e. vertex attributes)
glBufferData(GL_ARRAY_BUFFER, sizeof(vertices), vertices, GL_STATIC_DRAW)   # assign vertex data to the buffer

# Vertex Array Object (VAO)
vao = Ref(GLuint(0))
glGenVertexArrays(1, vao)    # generate 1 vertex array object names
glBindVertexArray(vao[])     # bind the vertex array object

# Link vertex data to attributes
pos_attribute = glGetAttribLocation(shader_program, "position")    # get the location of the variable "position" in the shader_program
glVertexAttribPointer(pos_attribute, 2, GL_FLOAT, GL_FALSE, 0, C_NULL) # define location and data format of the position attribute data
glEnableVertexAttribArray(pos_attribute)    # enable the vertex attribute array

# background color
glClearColor(0.2f0, 0.3f0, 0.3f0, 1.0f0)

The event loop is extended by the function call `glDrawArrays(GL_TRIANGLES, 0, 3)`. Here the parameters specify:
* what kind of primitives to render (`GL_TRIANGLES`).
* the starting index (0)
* the number of vertices to be rendered (3)

In [16]:
# Draw nothing, while waiting for a close event
while !GLFW.WindowShouldClose(window)
    
    # Render here
    glClear(GL_COLOR_BUFFER_BIT)
    glDrawArrays(GL_TRIANGLES, 0, 3)
    
    # Swap front and back buffers
    GLFW.SwapBuffers(window)
    
    # Poll for and process events
    GLFW.PollEvents()    
end
GLFW.DestroyWindow(window)  # needed if you're running this from the REPL

Dict{GLFW.Window, Ref{Vector{Union{Nothing, Function}}}}()

## 3. Element Buffer Objects

For more complex objects, it does not make sense to repeat vertices for each primitive which they occur in. In order to specify the vertices associated with each primitives, one can use vertex element objects (EBOs).

Let us extend our code by an EBO and use it to draw a square.

The window setup and the shader program remain unchanged.

In [17]:
# setup window and OpenGL context
window = initWindow(title="Square", callbackFunc=key_callback)
# compile shader program
shader_program = compileShaderProgram(vertex_source, fragment_source)

0x00000003

The sequence to generate an EBO is exactly the same as the one used to generate VBOs. The only difference is that the buffer object needs to be bound to the target `GL_ELEMENT_BUFFER`. Moreover, the EBO needs to be created after creation of the current VAO.  

In [18]:
# The vertices of our triangle
vertices = Float32[0.5,0.5, 0.5,-0.5, -0.5,-0.5, -0.5,0.5]
indices = UInt32[0,1,3, 1,2,3]

vbo = Ref(GLuint(0))   # initial value is irrelevant, just allocate space
glGenBuffers(1, vbo)   # generate one Buffer object name and store it in vbo
glBindBuffer(GL_ARRAY_BUFFER, vbo[])   # bind the buffer to target GL_ARRAY_BUFFER (i.e. vertex attributes)
glBufferData(GL_ARRAY_BUFFER, sizeof(vertices), vertices, GL_STATIC_DRAW)   # assign vertex data to the buffer

# Vertex Array Object (VAO)
vao = Ref(GLuint(0))
glGenVertexArrays(1, vao)    # generate 1 vertex array object names
glBindVertexArray(vao[]) 

# Element Buffer object
ebo = Array{GLuint}(undef,1)
glGenBuffers(1, ebo)
glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, ebo[1])
glBufferData(GL_ELEMENT_ARRAY_BUFFER, sizeof(indices), indices, GL_STATIC_DRAW)

# Link vertex data to attributes
pos_attribute = glGetAttribLocation(shader_program, "position")    # get the location of the variable "position" in the shader_program
glVertexAttribPointer(pos_attribute, 2, GL_FLOAT, GL_FALSE, 0, C_NULL) # define location and data format of the position attribute data
glEnableVertexAttribArray(pos_attribute)    # enable the vertex attribute array

# background color
glClearColor(0.2f0, 0.3f0, 0.3f0, 1.0f0)

The event loop stays almost the same. Note however that the draw call needs to be modified for indexed data

In [19]:
# Draw nothing, while waiting for a close event
while !GLFW.WindowShouldClose(window)
    
    # Render here
    glClear(GL_COLOR_BUFFER_BIT)
    # Different draw call for indexed data
    glDrawElements(GL_TRIANGLES, 6, GL_UNSIGNED_INT, C_NULL)
    
    # Swap front and back buffers
    GLFW.SwapBuffers(window)
    
    # Poll for and process events
    GLFW.PollEvents()    
end
GLFW.DestroyWindow(window)  # needed if you're running this from the REPL

Dict{GLFW.Window, Ref{Vector{Union{Nothing, Function}}}}()

## 4. Uniforms and key-callbacks
An important concept in OpenGL are uniforms. These variables cannot change during the excecution of the shaders. However, they can be modified between shader executions.

Add a `uniform vec2 dxy` to the fragment shader which shifts the vertices by the given amount.
In order to link a given vector `Δxy` to the uniform you can use the following code
```
dxyLocation = glGetUniformLocation(shader_program, "dxy")
glUniform2f(dxyLocation, Δxy[1], Δxy[2])

```
Now modify the funcion `key_callack`, such that the image is shifted to the left/right/up/down by 0.1, when the corresponding left/right/up/down arrow keys are pressed. The relevant key codes are `GLFW.KEY_LEFT`, `GLFW.KEY_RIGHT`, `GLFW.KEY_UP`and `GLFW.KEY_DOWN`. 

*Note:* when modifying a global variable `Δxy` from inside a function, the assignment must be prefixed with the keyword `global`


In [20]:
function key_callback(window::GLFW.Window, key::GLFW.Key, scancode::Int32, action::GLFW.Action, mods::Int32)
    # close window when the esc-key is pressed
    if (key == GLFW.KEY_ESCAPE && action == GLFW.PRESS)
        GLFW.SetWindowShouldClose(window, true)
    # movements of the object
    elseif (key == GLFW.KEY_RIGHT && action == GLFW.PRESS)
        global Δxy .+= [0.1f0, 0.0f0]
        dxyLocation = glGetUniformLocation(shader_program, "dxy")
        # glUniform2fv(dxyLocation, 1, Δxy)
        glUniform2f(dxyLocation, Δxy[1], Δxy[2])
    elseif (key == GLFW.KEY_LEFT && action == GLFW.PRESS)
        global Δxy .-= [0.1f0, 0.0f0]
        dxyLocation = glGetUniformLocation(shader_program, "dxy")
        # glUniform2fv(dxyLocation, 1, Δxy)
        glUniform2f(dxyLocation, Δxy[1], Δxy[2])
    elseif (key == GLFW.KEY_UP && action == GLFW.PRESS)
        global Δxy .+= [0.0f0, 0.1f0]
        dxyLocation = glGetUniformLocation(shader_program, "dxy")
        # glUniform2fv(dxyLocation, 1, Δxy)
        glUniform2f(dxyLocation, Δxy[1], Δxy[2])
    elseif (key == GLFW.KEY_DOWN && action == GLFW.PRESS)
        global Δxy .-= [0.0f0, 0.1f0]
        dxyLocation = glGetUniformLocation(shader_program, "dxy")
        # glUniform2fv(dxyLocation, 1, Δxy)
        glUniform2f(dxyLocation, Δxy[1], Δxy[2])
    end
end

# setup window and OpenGL context
window = initWindow(title="Square", callbackFunc=key_callback)

### implement vertex and fragment shader in GLSL (OpenGL Shading Language) ###

# The vertex shader
vertex_source = """
#version 150
in vec2 position;
uniform vec2 dxy;

void main()
{
    gl_Position = vec4(position+dxy, 0.0, 1.0);
}
"""

# The fragment shader
fragment_source = """
# version 150
out vec4 outColor;
void main()
{
    outColor = vec4(0.176, 0.776, 0.839, 1.0);
}
"""

# compile shader program
shader_program = compileShaderProgram(vertex_source, fragment_source)

# The vertices of our triangle
vertices = Float32[0.5,0.5,0.5,-0.5,-0.5,-0.5,-0.5,0.5]
indices = UInt32[0,1,3,1,2,3]

vbo = Ref(GLuint(0))   # initial value is irrelevant, just allocate space
glGenBuffers(1, vbo)   # generate one Buffer object name and store it in vbo
glBindBuffer(GL_ARRAY_BUFFER, vbo[])   # bind the buffer to target GL_ARRAY_BUFFER (i.e. vertex attributes)
glBufferData(GL_ARRAY_BUFFER, sizeof(vertices), vertices, GL_STATIC_DRAW)   # assign vertex data to the buffer

# Vertex Array Object (VAO)
vao = Ref(GLuint(0))
glGenVertexArrays(1, vao)    # generate 1 vertex array object names
glBindVertexArray(vao[]) 

# Element Buffer object
ebo = Array{GLuint}(undef,1)
glGenBuffers(1, ebo)
glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, ebo[1])
glBufferData(GL_ELEMENT_ARRAY_BUFFER, sizeof(indices), indices, GL_STATIC_DRAW)

# Link vertex data to attributes
pos_attribute = glGetAttribLocation(shader_program, "position")    # get the location of the variable "position" in the shader_program
glVertexAttribPointer(pos_attribute, 2, GL_FLOAT, GL_FALSE, 0, C_NULL) # define location and data format of the position attribute data
glEnableVertexAttribArray(pos_attribute)    # enable the vertex attribute array

# background color
glClearColor(0.2f0, 0.3f0, 0.3f0, 1.0f0)

# uniform describing the shift
Δxy = [0.0f0, 0.0f0]
dxyLocation = glGetUniformLocation(shader_program, "dxy")
# glUniform2fv(dxyLocation, 1,Δxy)
glUniform2f(dxyLocation, Δxy[1], Δxy[2])

# Draw nothing, while waiting for a close event
while !GLFW.WindowShouldClose(window)
    
    # Render here
    glClear(GL_COLOR_BUFFER_BIT)
    # Different draw call for indexed data
    glDrawElements(GL_TRIANGLES, 6, GL_UNSIGNED_INT, C_NULL)
    
    # Swap front and back buffers
    GLFW.SwapBuffers(window)
    
    # Poll for and process events
    GLFW.PollEvents()    
end
GLFW.DestroyWindow(window)  # needed if you're running this from the REPL

Dict{GLFW.Window, Ref{Vector{Union{Nothing, Function}}}}()